# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка библиотек

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
pd.options.mode.chained_assignment = None

## Загрузка и подготовка данных

In [2]:
data_0 = pd.read_csv('/Users/daniilsobolev/Desktop/ml projects/ML в бизнесе/geo_data_0.csv')
data_1 = pd.read_csv('/Users/daniilsobolev/Desktop/ml projects/ML в бизнесе/geo_data_1.csv')
data_2 = pd.read_csv('/Users/daniilsobolev/Desktop/ml projects/ML в бизнесе/geo_data_2.csv')

In [3]:
display(data_0.sample(3))

,id,f0,f1,f2,product
39031,p4fp3,0.692938,0.733696,0.819374,54.727979
50199,tiI5f,0.373257,1.019321,5.816986,145.022819
31279,mRbA5,1.576501,-0.517157,6.568617,133.555725


In [4]:
data_0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [5]:
display(data_1.sample(3))

,id,f0,f1,f2,product
83646,cyz0A,17.422562,1.922911,3.994248,107.813044
44609,1vgZ3,-8.182322,-1.950629,1.005105,30.132364
83689,YPmJB,-9.794835,-13.504305,0.997823,30.132364


In [6]:
data_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [7]:
display(data_2.sample(3))

,id,f0,f1,f2,product
91396,jkfzV,-1.673600,1.610521,-0.639270,107.124538
63243,U9hLI,0.819895,2.714162,3.364924,165.718428
71152,9oZiR,-0.133357,0.823229,6.216855,35.716360


In [8]:
data_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [9]:
print(data_0.duplicated().sum())
print(data_1.duplicated().sum())
print(data_2.duplicated().sum())

0
0
0


### Вывод:

В каждой таблице четыре признака, один из которых целевой - product. 

Целевой признак - количественный, следовательно, решаем задачу регрессии.

Данные без пропусков, без дубликатов.

## Подготвка данных к обучению

В каждом наборе данных у нас есть лишний столбец - id, удалим его.

Отделим целевой признак в отдельную таблицу. 

Разделим данные на тестовую и валидационную выборки в соотношении 3:1.

После проведем масштабирование - стандартизацию признаков.

### DATA 0

In [10]:
data_0 = data_0.drop(['id'], axis=1)

In [11]:
features_0 = data_0.drop(['product'], axis=1)
target_0 = data_0['product']

In [12]:
features_train_0, features_valid_0, target_train_0, target_valid_0 = train_test_split(features_0, target_0, 
                                                                              test_size=0.25, 
                                                                              random_state=12345)

In [13]:
numeric = ['f0', 'f1', 'f2']

scaler = StandardScaler()
scaler.fit(features_train_0[numeric])

features_train_0[numeric] = pd.DataFrame(scaler.transform(features_train_0[numeric]), 
                                       features_train_0[numeric].index, 
                                       features_train_0[numeric].columns)

features_valid_0[numeric] = pd.DataFrame(scaler.transform(features_valid_0[numeric]), 
                                       features_valid_0[numeric].index, 
                                       features_valid_0[numeric].columns)

### DATA 1

In [14]:
data_1 = data_1.drop(['id'], axis=1)

In [15]:
features_1 = data_1.drop(['product'], axis=1)
target_1 = data_1['product']

In [16]:
features_train_1, features_valid_1, target_train_1, target_valid_1 = train_test_split(features_1, target_1, 
                                                                              test_size=0.25, 
                                                                              random_state=12345)

In [17]:
scaler = StandardScaler()
scaler.fit(features_train_0[numeric])

features_train_1[numeric] = pd.DataFrame(scaler.transform(features_train_1[numeric]), 
                                       features_train_1[numeric].index, 
                                       features_train_1[numeric].columns)

features_valid_1[numeric] = pd.DataFrame(scaler.transform(features_valid_1[numeric]), 
                                       features_valid_1[numeric].index, 
                                       features_valid_1[numeric].columns)

### DATA 2

In [18]:
data_2 = data_2.drop(['id'], axis=1)

In [19]:
features_2 = data_2.drop(['product'], axis=1)
target_2 = data_2['product']

In [20]:
features_train_2, features_valid_2, target_train_2, target_valid_2 = train_test_split(features_2, target_2, 
                                                                              test_size=0.25, 
                                                                              random_state=12345)

In [21]:
scaler = StandardScaler()
scaler.fit(features_train_0[numeric])

features_train_2[numeric] = pd.DataFrame(scaler.transform(features_train_2[numeric]), 
                                       features_train_2[numeric].index, 
                                       features_train_2[numeric].columns)

features_valid_2[numeric] = pd.DataFrame(scaler.transform(features_valid_2[numeric]), 
                                       features_valid_2[numeric].index, 
                                       features_valid_2[numeric].columns)

## Обучение и проверка модели

### DATA 0

In [22]:
model = LinearRegression()
model.fit(features_train_0, target_train_0)

prediction_0 = model.predict(features_valid_0)
mse = mean_squared_error(target_valid_0, prediction_0)
print('RMSE:', mse ** 0.5)
print('MAE:', mean_absolute_error(target_valid_0, prediction_0))

print('Среднее значение предсказаний:', prediction_0.mean())
print('Среднее значение ответов:', target_valid_0.mean())

RMSE: 37.5794217150813
MAE: 30.919600777151313
Среднее значение предсказаний: 92.59256778438035
Среднее значение ответов: 92.07859674082941


### DATA 1

In [23]:
model = LinearRegression()
model.fit(features_train_1, target_train_1)

prediction_1 = model.predict(features_valid_1)
mse = mean_squared_error(target_valid_1, prediction_1)
print('RMSE:', mse ** 0.5)
print('MAE:', mean_absolute_error(target_valid_1, prediction_1))

print('Среднее значение предсказаний:', prediction_1.mean())
print('Среднее значение ответов:', target_valid_1.mean())

RMSE: 0.893099286775617
MAE: 0.7187662442124758
Среднее значение предсказаний: 68.728546895446
Среднее значение ответов: 68.72313602437494


### DATA 2

In [24]:
model = LinearRegression()
model.fit(features_train_2, target_train_2)

prediction_2 = model.predict(features_valid_2)
mse = mean_squared_error(target_valid_2, prediction_2)
print('RMSE:', mse ** 0.5)
print('MAE:', mean_absolute_error(target_valid_2, prediction_2))

print('Среднее значение предсказаний:', prediction_2.mean())
print('Среднее значение ответов:', target_valid_2.mean())

RMSE: 40.02970873393434
MAE: 32.792652105481814
Среднее значение предсказаний: 94.96504596800489
Среднее значение ответов: 94.88423280885489


### Вывод:

В каждом датасете среднее значение предсказаний слабо отличается от среднего значения ответов. Но среднеквадртаичное отклонение в нулевом и во втором, достигает отметки в районе 35-40. 

А в первом в районе около 1. То есть в этом случае модель лучше предсказывает.

## Расчёт прибыли и рисков

### Достаточный объём сырья для безубыточной разработки:

In [25]:
BUDGET = 10000000000 #10 млрд
ONE_BURREL = 450000 #450 тыс
COUNT = 200 # количество скважин

volume = BUDGET / ONE_BURREL / COUNT
print('Достаточный объём сырья для одной скважины:', volume)

Достаточный объём сырья для одной скважины: 111.11111111111111


In [26]:
print('Средний объём в нулевом регионе:', prediction_0.mean())
print('Средний объём в первом регионе:', prediction_1.mean())
print('Средний объём во втором регионе:', prediction_2.mean())

Средний объём в нулевом регионе: 92.59256778438035
Средний объём в первом регионе: 68.728546895446
Средний объём во втором регионе: 94.96504596800489


#### Вывод

Средний объём сырья в одной скважине в каждом из регионов меньше, чем объём необходимый для безубыточной разработки новых скважин. Следовательно, не стоит разрабатывать все скважины в регионе, будем отбирать их по лучшим предсказаниям.

### Функция для подсчета прибыли:

In [27]:
def revenue(target, probabilities, count):
    probs_sorted = probabilities.sort_values(ascending=False)
    selected = target[probs_sorted.index][:count]
    return 450 * selected.sum() - 10000000

### DATA 0

In [28]:
target = target_valid_0
probabilities = pd.Series(prediction_0, index=target.index)

In [29]:
state = np.random.RandomState(12345)
values = []
count = 0

for i in range(1000):
    target_subsample = target.sample(n=500, replace=True, random_state=state)
    probs_subsample = probabilities[target_subsample.index]
    
    values.append(revenue(target_subsample, probs_subsample, 200))
    
    if revenue(target_subsample, probs_subsample, 200) < 0:
        count += 1

In [30]:
values = pd.Series(values)
lower = values.quantile(0.025)
upper = values.quantile(0.975)

print('Риск уйти в убыток:', count/1000 * 100, '%')

mean = values.mean()
print("Средняя прибыль:", int(mean), 'тыс.')
print("2.5% нижний квантиль:", int(lower), 'тыс.')
print("2.5% верхний квантиль:", int(upper), 'тыс.')

Риск уйти в убыток: 6.0 %
Средняя прибыль: 425938 тыс.
2.5% нижний квантиль: -102090 тыс.
2.5% верхний квантиль: 947976 тыс.


### DATA 1

In [31]:
target = target_valid_1
probabilities = pd.Series(prediction_1, index=target.index)

In [32]:
state = np.random.RandomState(12345)
values = []
count = 0

for i in range(1000):
    target_subsample = target.sample(n=500, replace=True, random_state=state)
    probs_subsample = probabilities[target_subsample.index]
    
    values.append(revenue(target_subsample, probs_subsample, 200))
    
    if revenue(target_subsample, probs_subsample, 200) < 0:
        count += 1

In [33]:
values = pd.Series(values)
lower = values.quantile(0.025)
upper = values.quantile(0.975)

print('Риск уйти в убыток:', count/1000 * 100, '%')

mean = values.mean()
print("Средняя прибыль:", int(mean), 'тыс.')
print("2.5% нижний квантиль:", int(lower), 'тыс.')
print("2.5% верхний квантиль:", int(upper), 'тыс.')

Риск уйти в убыток: 1.0 %
Средняя прибыль: 515222 тыс.
2.5% нижний квантиль: 68873 тыс.
2.5% верхний квантиль: 931547 тыс.


### DATA 2

In [34]:
target = target_valid_2
probabilities = pd.Series(prediction_2, index=target.index)

In [35]:
state = np.random.RandomState(12345)
values = []
count = 0

for i in range(1000):
    target_subsample = target.sample(n=500, replace=True, random_state=state)
    probs_subsample = probabilities[target_subsample.index]
    
    values.append(revenue(target_subsample, probs_subsample, 200))
    
    if revenue(target_subsample, probs_subsample, 200) < 0:
        count += 1

In [36]:
values = pd.Series(values)
lower = values.quantile(0.025)
upper = values.quantile(0.975)

print('Риск уйти в убыток:', count/1000 * 100, '%')

mean = values.mean()
print("Средняя прибыль:", int(mean), 'тыс.')
print("2.5% нижний квантиль:", int(lower), 'тыс.')
print("2.5% верхний квантиль:", int(upper), 'тыс.')

Риск уйти в убыток: 6.4 %
Средняя прибыль: 435008 тыс.
2.5% нижний квантиль: -128880 тыс.
2.5% верхний квантиль: 969706 тыс.


### Вывод:

Безубыточная работа с вероятностью убытков меньше 2.5% только в первом регионе(data 1). Поэтому для разработки скважин следует выбрать именно его.

## Общий вывод

В ходе проекта были изучены три датасета - пробы нефти в трёх регионах. Также была построена модель для определния региона, где добыча принесет наибольшую прибыль. А также проанализированы риски и прибыль каждого региона.<br>

В первую очередь данные были подготовлены к обучению: 
- были удалены лишние признаки, 
- данные разделены на целевой и другие признаки, 
- данные разделены на обучающую и валидациаонную выборки в соотношении 3:1,
- масштабированы путем стандартизации данных

Далее рассматривался алгоритм линейной регрессии, так как он лучше предсказывает в подобных ситуциях:
- были обучены модели,
- сделаны предсказания, 
- рассчитаны средний запас предсказанного сырья и метрика RMSE
Было получено, что на нулевом и втором датасетах модель плохо предсказывала, так как была большая метрика RMSE (-40), а на первом маленькая (-1), но средние значения показаний, наоборот, на первом датасете были меньше (-70), на других (-95).


Был рассчитан достаточный объём сырья для каждой скважины, были сделаны выводы, если будут исследоваться все скважины в регионе, но нефтедобывающая компания уйдет в убыток. 

В заключении были рассчитаны риски уйти в убыток для каждого региона. Требования компании были в том, чтобы вероятность убытков была меньше 2,5%, таким требованиям удовлетворял только первый регион (вероятность уйти в убыток - 1%). Средняя прибыль в этом регионе может составить 515 222 тыс. рублей.
